# Facebook Ego Network Analysis

This notebook contains two visualizations using the Facebook ego network dataset.

In [ ]:
import networkx as nx
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

# Load the dataset
url = "https://snap.stanford.edu/data/facebook_combined.txt"
G = nx.read_edgelist(url)

# Create spring layout positions
pos = nx.spring_layout(G, seed=42)

# Create node dataframe
nodes_df = pd.DataFrame({
    'node': list(pos.keys()),
    'x': [pos[n][0] for n in pos],
    'y': [pos[n][1] for n in pos],
    'degree': [G.degree(n) for n in pos]
})

# Create edge dataframe
edges_df = pd.DataFrame([
    {'source': u, 'target': v, 'x1': pos[u][0], 'y1': pos[u][1], 'x2': pos[v][0], 'y2': pos[v][1]}
    for u, v in G.edges()
])

## Visualization 1: Network Graph of Facebook Ego Network

In [ ]:
# Create the edges as lines
edges = alt.Chart(edges_df).mark_line(opacity=0.05).encode(
    x='x1', y='y1', x2='x2', y2='y2'
)

# Create the nodes as circles
nodes = alt.Chart(nodes_df).mark_circle(size=30).encode(
    x='x',
    y='y',
    color=alt.Color('degree:Q', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(title="Degree")),
    tooltip=['node', 'degree']
).properties(
    width=600,
    height=600,
    title="Facebook Ego Network: Force-Directed Layout"
)

network_chart = edges + nodes
network_chart

## Visualization 2: Interactive Degree Histogram

In [ ]:
# Prepare degree distribution dataframe
degree_df = pd.DataFrame(nodes_df['degree'].value_counts().sort_index()).reset_index()
degree_df.columns = ['degree', 'count']

# Create interactive histogram
selection = alt.selection_interval(encodings=['x'])

hist = alt.Chart(degree_df).mark_bar().encode(
    x=alt.X('degree:Q', bin=alt.Bin(maxbins=50), title='Degree (Number of Friends)'),
    y=alt.Y('count:Q', title='Number of Users'),
    color=alt.condition(selection, alt.value('steelblue'), alt.value('lightgray')),
    tooltip=['degree', 'count']
).add_params(
    selection
).properties(
    width=600,
    height=400,
    title="Degree Distribution of Facebook Users"
)

hist